# Modular

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import time
from nltk.corpus import wordnet
import re
import enchant

# Read data

In [47]:
filename = '/Users/Lyf/OneDrive/study/WISC/2017_spring/Stat_628/hw2/data/train_data.csv'
reviews = pd.read_csv(filename)

# Test number

In [69]:
N = 200

In [70]:
yelp = reviews[0:N].copy()
yelp_ori = yelp.copy

# 加上长度变量

In [71]:
yelp.loc[:, 'text length'] = yelp['text'].apply(len)

In [72]:
d = enchant.Dict("en_US")

# 改缩写

In [73]:
for i in range(N):
    sentence = yelp['text'][i]
    sentence2 = re.sub('n\'t', ' not', sentence)
    sentence3 = re.sub('\'s', ' is', sentence2)
    sentence4 = re.sub('\'ve', ' have', sentence3)
    sentence5 = re.sub('\'d', ' would', sentence4)
    sentence6 = re.sub('\'m', ' am', sentence5)
    sentence7 = re.sub('Ive', 'I have', sentence6)
    yelp.loc[i, 'text'] = sentence6
#     if(i%100 == 0):
#         print(i/100)

# Delete unEnglish

In [74]:
# t1 = time.time()
index = []
for i in range(N):
    sentence = yelp['text'][i]
    sentence4 = re.sub('[^a-zA-Z]',' ', sentence)
    words = sentence4.split()
    n = 0
    count = 0
    for j in range(len(words)):
        word = words[j]
        if d.check(word):
            count += 1
#         else:
#             print(word)
        n += 1
    if(n == 0):
#         print(i)
        index.append(i)
    else:
        if(count/n < 0.8):
            index.append(i)       
# time.time() - t1

In [75]:
yelp2 = yelp.drop(index)

# Get positive and negative

In [76]:
reviews_total = ''
for i in range(N):
    reviews_total = reviews_total+" "+yelp.loc[i, 'text']

In [77]:
reviews_total
positive_words = ["nice","prime","great","awesome","yummy","friendly","delicious",
                  "love","spicy","good","favorite","well","clean","enjoy"]
negetive_words = ["suck","terrible","amazing","messy","worse","hit","miss"]

In [87]:
print(reviews['text'][9])

I saw this restaurant on a whim when I was looking for a good indian food place to try and decided that this was the one. We opted for delivery because why not.

I usually judge the place based on my all time favourite Butter chicken and when I got it for sure this place has it down! It surpassed the other ones I tried and for sure this is where am getting it from now on.

Aside from that dish, we also got the chicken tikka masala and the garlic naan bread. The masala was alright, I tasted a better one elsewhere so this one wasn't really my favourite from them. And the garlic naan was yummy. Really generous with the garlic for sure. 

And I also got a Mango lassi, I didnt like their version that much so I think I'll skip on this one next time.

Anyway, I have yet to try their other dishes in the future so for now am just giving them three stars. And also the other reason for the rating was because our food arrived late. Am glad I waited a little bit more because I was close to complain

# Add not

In [66]:
def add_not(sentence):
    words = sentence.split().copy()
    not_index = 0
    for index, word in enumerate(words):
        if(not_index != 0):
            words[index] = 'not_' + words[index]
        if(re.match('.*[,.?!\'\"]$', word)):
            not_index = 0
        if(word == 'not'):
            not_index = 1
    #         print(index)
    sentence = " ".join(words)
    return(sentence)
for i in range(N):
    sentence = yelp.loc[i, 'text']
    sentence2 = add_not(sentence)
    yelp.loc[i, 'text'] = sentence2

# 停词，标点加去时态

In [30]:
stops = set(stopwords.words("english"))

from nltk.stem.wordnet import WordNetLemmatizer 
lem = WordNetLemmatizer()

from nltk.stem.porter import PorterStemmer 
stem = PorterStemmer()

def noise_remove(review):
    review = re.sub("[^a-zA-Z]", " ", review)
    review = review.lower().split()
    useful_review = [w for w in review if not w in stops]  
    return " ".join([stem.stem(w) for w in useful_review])


In [31]:
yelp3 = yelp2.copy()
for i in range(yelp3.shape[0]):
    yelp3.iloc[i, 2] = noise_remove(yelp2.iloc[i, 2])

In [32]:
yelp3.to_csv("first10000.csv", index = False)